# Введение в методы анализа данных. Язык Python.

## Лекция 11. Web Scraping
<br><br><br><br>
__Аксентьев Артем (akseart@ya.ru)__

__Ксемидов Борис (nstalker.anonim@yandex.ru)__
<br>

# Зачем необходимо?

- Не всегда готовые датасеты удовлетворяют нашим потребностям
- Иногда неудобно просматривать данные на сайте
- Хочется перенести данные в другое место

# Схема web-scraping
- Анализ веб страницы
- Загрузка html кода
- Парсинг данных
- Их обработка
- Создание какой-либо осознанной структуры данных

# Инструменты для web-scraping:
- Инструменты для парсинга html
    - bs4
    - lxml
- Инструменты для получения html страниц
    - requests
    - selenium

In [8]:
%%writefile index.html

<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.01//EN" "http://www.w3.org/TR/html4/strict.dtd">
<html>
    <head>
        <meta http-equiv="Content-Type" content="text/html; charset=utf-8">
        <title>Пример веб-страницы</title>
    </head>
    <body class="" id="" name="">
        <h1>Заголовок</h1>
        <div class="meta">
            <!-- Комментарий -->
            <p class="class">Первый абзац.</p>
        </div>
        <div>
            <p> Третий абзац</p>
        </div>
        <h1>Заголовок 1</h1>
        <p>Второй абзац.</p>
        <h1>Заголовок 2</h1>
    </body>
</html>

Writing index.html


## BeautifulSoup

In [9]:
from bs4 import BeautifulSoup
with open("index.html", "r") as f:
    text = f.read()
soup = BeautifulSoup(text, "html.parser")

In [10]:
soup.find("h1")

<h1>Заголовок</h1>

In [11]:
soup.find_all("h1")

[<h1>Заголовок</h1>, <h1>Заголовок 1</h1>, <h1>Заголовок 2</h1>]

In [12]:
soup.find("p", class_="class").text

'Первый абзац.'

In [13]:
soup.find("p", class_="class").parent

<div class="meta">
<!-- Комментарий -->
<p class="class">Первый абзац.</p>
</div>

In [18]:
list(soup.find("p", class_="class").parent.children)

['\n', ' Комментарий ', '\n', <p class="class">Первый абзац.</p>, '\n']

## lxml

In [19]:
from lxml import etree

tree = etree.parse("index.html", etree.HTMLParser())

In [20]:
tree.xpath("//p")

[<Element p at 0x105c28800>,
 <Element p at 0x10575ff00>,
 <Element p at 0x105885200>]

In [21]:
tree.xpath("//p/text()")

['Первый абзац.', ' Третий абзац', 'Второй абзац.']

In [22]:
tree.xpath('//p[@class="class"]/text()')

['Первый абзац.']

In [23]:
tree.xpath('//div/p/text()')

['Первый абзац.', ' Третий абзац']

## Selenium

In [24]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities

desired_capabilities = DesiredCapabilities().SAFARI
desired_capabilities["pageLoadStrategy"] = "eager"
driver = webdriver.Safari(desired_capabilities=desired_capabilities)

driver.get("https://mai.ru/education/studies/schedule/ppc.php?guid=f25361dd-1d99-11e0-9baf-1c6f65450efa&week=8")
first_form_input = driver.find_element_by_class_name("mb-4")
print("OK")

OK


In [ ]:
first_form_input = driver.find_element(By.CLASS_NAME,  "mb-4")

In [25]:
first_form_input.text

'\n\t\t\t\t\tЧт,\xa020\xa0октября\t\t\t\t'

In [30]:
first_form_input.parent.find_element_by_class_name("mb-4").text

'\n\t\t\t\t\tЧт,\xa020\xa0октября\t\t\t\t'

In [31]:
driver.find_elements_by_xpath("//div/text()")

[<selenium.webdriver.remote.webelement.WebElement (session="19680F18-A5F2-4883-B029-79FDE2482C72", element="node-51FD3CF7-8560-4ED1-9A29-87C9D1AFCA75")>,
 <selenium.webdriver.remote.webelement.WebElement (session="19680F18-A5F2-4883-B029-79FDE2482C72", element="node-8A178B47-99E9-4FAC-9E3F-68876727451B")>,
 <selenium.webdriver.remote.webelement.WebElement (session="19680F18-A5F2-4883-B029-79FDE2482C72", element="node-F3D84CDE-3739-4BF5-AB44-180F0C7FEFB0")>,
 <selenium.webdriver.remote.webelement.WebElement (session="19680F18-A5F2-4883-B029-79FDE2482C72", element="node-66FB97CF-655F-4796-B198-8F9918599895")>,
 <selenium.webdriver.remote.webelement.WebElement (session="19680F18-A5F2-4883-B029-79FDE2482C72", element="node-591EEFCF-7842-48E5-B9FD-BE7AB4A9FCCA")>,
 <selenium.webdriver.remote.webelement.WebElement (session="19680F18-A5F2-4883-B029-79FDE2482C72", element="node-79EBFAC5-EBAF-4B59-B96B-6C37FF55E4DA")>,
 <selenium.webdriver.remote.webelement.WebElement (session="19680F18-A5F2-48

In [32]:
driver.save_screenshot('image.png')

True

In [37]:
driver.quit()

InvalidSessionIdException: Message: 


![](image.png)

## Requests

In [35]:
import requests

page = requests.get("https://mai.ru/education/studies/schedule/ppc.php?guid=f25361dd-1d99-11e0-9baf-1c6f65450efa&week=8", verify=False)

/Users/artem/miniconda3/envs/Python_Lections_MSU/lib/python3.8/site-packages/urllib3/connectionpool.py:1043: InsecureRequestWarning: Unverified HTTPS request is being made to host 'mai.ru'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  


In [ ]:
page.text

In [38]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.9; rv:45.0) Gecko/20100101 Firefox/45.0'
}
page = requests.get("https://mai.ru/education/studies/schedule/ppc.php?guid=f25361dd-1d99-11e0-9baf-1c6f65450efa&week=8", headers=headers, verify=False)

/Users/artem/miniconda3/envs/Python_Lections_MSU/lib/python3.8/site-packages/urllib3/connectionpool.py:1043: InsecureRequestWarning: Unverified HTTPS request is being made to host 'mai.ru'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  


In [39]:
page

<Response [200]>

In [ ]:
page.text

# Рекомендуемая литература:
- http://wiki.python.su/Документации/BeautifulSoup
- https://www.crummy.com/software/BeautifulSoup/bs4/doc.ru/bs4ru.html
- http://htmlbook.ru/samhtml

# Вопросы для самостоятельного изучения:
- "Общение" с веб страницей с помощью Selenium(обработка полей ввода и тд)
- Более глубоко изучить понравившуюся библиотеку (bs4/lxml)
- HTTP запросы и как они устроены
- HTML(на базовом уровне)

# Вопросы к зачету

- Зачем нужен web-scraping?
- Какие средства для web-scrapinga используются